In [65]:
import pandas as pd

# Load datasets
stores_df = pd.read_csv("stores.csv")
features_df = pd.read_csv("features.csv")
train_df = pd.read_csv("train.csv")

# Preview
print("📁 stores.csv")
display(stores_df.head())

print("\n📁 features.csv")
display(features_df.head())

print("\n📁 train.csv")
display(train_df.head())


📁 stores.csv


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875



📁 features.csv


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False



📁 train.csv


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [85]:
features_df = pd.read_csv("features.csv")
features_df['Date'] = features_df['Date'].astype(str).str.strip()
features_df['Date'] = pd.to_datetime(features_df['Date'], errors='coerce')
print("Missing date:", features_df['Date'].isnull().sum())
print("Date range:", features_df['Date'].min(), "to", features_df['Date'].max())
print(features_df['Date'].dtype)

Missing date: 0
Date range: 2010-02-05 00:00:00 to 2013-07-26 00:00:00
datetime64[ns]


In [86]:
#step 1: Convert 'Date' column to datetime format
train_df = pd.read_csv("train.csv")
train_df['Date'] = pd.to_datetime(train_df['Date'], errors='coerce')

print(train_df['Date'].isnull().sum())

0


In [87]:
#Step 2: Fill missing values 
# Fill Markdown columns with 0 (assuming no markdown means no discount)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
features_df[markdown_cols] = features_df[markdown_cols].fillna(0)

#Sort before forward fill/backward fill
features_df.sort_values(by=['Store', 'Date'], inplace=True)

# Forward fill missing values for 'CPI', 'Unemployment'
features_df['CPI'] = features_df.groupby('Store')['CPI'].ffill().bfill()
features_df['Unemployment'] = features_df.groupby('Store')['Unemployment'].ffill().bfill()

#Check for any remaining missing values
print("\nRemaining missing values in features_df:")
print(features_df.isnull().sum())



Remaining missing values in features_df:
Store           0
Date            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday       0
dtype: int64


In [88]:
print("Missing values in train_df:")
print(train_df.isnull().sum())

Missing values in train_df:
Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64


In [89]:
# Merging datasets
# Step 1: Merge train with features on Store and Date
merged_df = pd.merge(train_df, features_df, on=['Store', 'Date'], how='left')

# Step 2: Merge with stores on Store
merged_df = pd.merge(merged_df, stores_df, on='Store', how='left')

# Step 3: Check for any remaining missing values
print("\nRemaining missing values after merging:")
print(merged_df.isnull().sum())

# preview merged dataset
merged_df.head()



Remaining missing values after merging:
Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday_x     0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday_y     0
Type            0
Size            0
dtype: int64


,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,True,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,False,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,False,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,False,A,151315


In [91]:
#add columns for year, month, and day
merged_df['Year'] = merged_df['Date'].dt.year
merged_df['Month'] = merged_df['Date'].dt.month
merged_df['Month_Name'] = merged_df['Date'].dt.strftime('%B')
merged_df['Week'] = merged_df['Date'].dt.isocalendar().week
merged_df['Day'] = merged_df['Date'].dt.day
merged_df['DayOfWeek'] = merged_df['Date'].dt.day_of_week
merged_df['Day_Name'] = merged_df['Date'].dt.strftime('%A')

merged_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,IsHoliday_y,Type,Size,Year,Month,Month_Name,Week,Day,DayOfWeek,Day_Name
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,False,A,151315,2010,2,February,5,5,4,Friday
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,...,True,A,151315,2010,2,February,6,12,4,Friday
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,...,False,A,151315,2010,2,February,7,19,4,Friday
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,...,False,A,151315,2010,2,February,8,26,4,Friday
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,...,False,A,151315,2010,3,March,9,5,4,Friday


In [ ]:
# Define holiday dates
superbowl_dates = pd.to_datetime(['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08'])
laborday_dates = pd.to_datetime(['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'])
thanksgiving_dates = pd.to_datetime(['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'])
christmas_dates = pd.to_datetime([
    '2010-12-23', '2010-12-24', '2010-12-31',
    '2011-12-23', '2011-12-24', '2011-12-30',
    '2012-12-24', '2012-12-28',
    '2013-12-27'
])

# Add holiday indicator columns
merged_df['IsSuperBowl'] = merged_df['Date'].isin(superbowl_dates).astype('object')
merged_df['IsLaborDay'] = merged_df['Date'].isin(laborday_dates).astype('object')
merged_df['IsThanksgiving'] = merged_df['Date'].isin(thanksgiving_dates).astype('object')
merged_df['IsChristmas'] = merged_df['Date'].isin(christmas_dates).astype('object')


In [94]:
merged_df.to_csv("merged_data.csv", index=False)